In [420]:
#imports
#numpy,pandas,scipy,math,matplotlib
import numpy as np
import pandas as pd
import scipy
from math import sqrt
import matplotlib.pyplot as plt

#estimators
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn import linear_model

#model metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score

#cross validation
from sklearn.model_selection import train_test_split

#math
from math import sqrt

In [421]:
#data
rawData = pd.read_csv('Credit_2.csv')
rawData.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,1,20000,female,university,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,default
1,2,120000,female,university,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,default
2,3,90000,female,university,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,not default
3,4,50000,female,university,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,not default
4,5,50000,male,university,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,not default


In [422]:
rawData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   ID                          30000 non-null  int64 
 1   LIMIT_BAL                   30000 non-null  int64 
 2   SEX                         30000 non-null  object
 3   EDUCATION                   30000 non-null  object
 4   MARRIAGE                    30000 non-null  int64 
 5   AGE                         30000 non-null  int64 
 6   PAY_0                       30000 non-null  int64 
 7   PAY_2                       30000 non-null  int64 
 8   PAY_3                       30000 non-null  int64 
 9   PAY_4                       30000 non-null  int64 
 10  PAY_5                       30000 non-null  int64 
 11  PAY_6                       30000 non-null  int64 
 12  BILL_AMT1                   30000 non-null  int64 
 13  BILL_AMT2                   30000 non-null  in

In [423]:
#dealing with non-numeric data- one-hot encoding
#converting all object variables to binary featues that one 0 and 1 classes 
rawData = pd.get_dummies(rawData)
print (rawData)


          ID  LIMIT_BAL  MARRIAGE  AGE  PAY_0  PAY_2  PAY_3  PAY_4  PAY_5  \
0          1      20000         1   24      2      2     -1     -1     -2   
1          2     120000         2   26     -1      2      0      0      0   
2          3      90000         2   34      0      0      0      0      0   
3          4      50000         1   37      0      0      0      0      0   
4          5      50000         1   57     -1      0     -1      0      0   
...      ...        ...       ...  ...    ...    ...    ...    ...    ...   
29995  29996     220000         1   39      0      0      0      0      0   
29996  29997     150000         2   43     -1     -1     -1     -1      0   
29997  29998      30000         2   37      4      3      2     -1      0   
29998  29999      80000         1   41      1     -1      0      0      0   
29999  30000      50000         1   46      0      0      0      0      0   

       PAY_6  ...  PAY_AMT5  PAY_AMT6  SEX_female  SEX_male  \
0         -2

In [424]:
rawData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 30 columns):
 #   Column                                  Non-Null Count  Dtype
---  ------                                  --------------  -----
 0   ID                                      30000 non-null  int64
 1   LIMIT_BAL                               30000 non-null  int64
 2   MARRIAGE                                30000 non-null  int64
 3   AGE                                     30000 non-null  int64
 4   PAY_0                                   30000 non-null  int64
 5   PAY_2                                   30000 non-null  int64
 6   PAY_3                                   30000 non-null  int64
 7   PAY_4                                   30000 non-null  int64
 8   PAY_5                                   30000 non-null  int64
 9   PAY_6                                   30000 non-null  int64
 10  BILL_AMT1                               30000 non-null  int64
 11  BILL_AMT2      

In [425]:
df = rawData.loc[rawData['default payment next month_not default']==1]
#assigning region as 1
df.head()
#need to encourage - limit bal >

,ID,LIMIT_BAL,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,...,PAY_AMT5,PAY_AMT6,SEX_female,SEX_male,EDUCATION_graduate school,EDUCATION_high school,EDUCATION_other,EDUCATION_university,default payment next month_default,default payment next month_not default
2,3,90000,2,34,0,0,0,0,0,0,...,1000,5000,1,0,0,0,0,1,0,1
3,4,50000,1,37,0,0,0,0,0,0,...,1069,1000,1,0,0,0,0,1,0,1
4,5,50000,1,57,-1,0,-1,0,0,0,...,689,679,0,1,0,0,0,1,0,1
5,6,50000,2,37,0,0,0,0,0,0,...,1000,800,0,1,1,0,0,0,0,1
6,7,500000,2,29,0,0,0,0,0,0,...,13750,13770,0,1,1,0,0,0,0,1


In [438]:
X=df[['MARRIAGE', 'AGE', 'SEX_female', 'SEX_male','EDUCATION_graduate school','EDUCATION_high school','EDUCATION_university']]
print('Summary of feature sample')
#X=df[['SEX_female', 'SEX_male','EDUCATION_graduate school','EDUCATION_high school','EDUCATION_university']]
#print('Summary of feature sample')
X.head()

Summary of feature sample


,MARRIAGE,AGE,SEX_female,SEX_male,EDUCATION_graduate school,EDUCATION_high school,EDUCATION_university
2,2,34,1,0,0,0,1
3,1,37,1,0,0,0,1
4,1,57,0,1,0,0,1
5,2,37,0,1,1,0,0
6,2,29,0,1,1,0,0


In [439]:
#features
#X=df1
#print('Summary of feature sample')
#X.head()

In [440]:
## dependent variable
#what we are trying to predict - #of 
#how to incorporate default/non default into this 
#df.loc[df[2]]
y = df.loc[:,'LIMIT_BAL']
#y = df.iloc['LIMIT_BAL']
y.head()
#does this need to be in a 1d array

2     90000
3     50000
4     50000
5     50000
6    500000
Name: LIMIT_BAL, dtype: int64

In [441]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = .30, random_state = 123)
X_train.shape, X_test.shape

((16354, 7), (7010, 7))

In [442]:
X_train_count = len(X_train.index)
print('The number of observations in the feature training set are:',str(X_train_count))
X_train.head(10)

The number of observations in the feature training set are: 16354


,MARRIAGE,AGE,SEX_female,SEX_male,EDUCATION_graduate school,EDUCATION_high school,EDUCATION_university
2665,2,22,1,0,0,0,1
25019,1,65,0,1,0,1,0
6699,2,43,0,1,0,0,1
27881,1,27,1,0,1,0,0
25361,1,26,1,0,0,0,1
15609,2,27,1,0,1,0,0
24547,1,41,0,1,1,0,0
29628,2,29,0,1,0,0,1
12830,1,38,1,0,0,0,1
19388,1,38,1,0,0,0,1


In [443]:
##why are # of observations the same 
y_train_count = len(y_train.index)
print('The number of observations in the y training set are:',str(y_train_count))
y_train.head()

The number of observations in the y training set are: 16354


2665      50000
25019    220000
6699      60000
27881    250000
25361    180000
Name: LIMIT_BAL, dtype: int64

<b> Random Forest Regression Model Fitting</b>

<b>Linear Regression Model Fitting</b>

In [444]:
#algosClass.append(('Linear Regression',LinearRegression()))

<b>Support Vector Regression Model Fitting</b>

In [445]:
#algosClass.append(('Support Vector Regression',SVR()))

In [446]:
# =[] creating empty list
#append is adding element to the list
#creating empty list and then adding the algos
algosClass = []
algosClass.append(('Random Forest Regressor',RandomForestRegressor()))
algosClass.append(('Linear Regression',LinearRegression()))
algosClass.append(('Support Vector Regression',SVR()))

In [447]:
#regression
results = []
names = []
for name, model in algosClass:
 result = cross_val_score(model, X,y, cv=3, scoring='r2')
 names.append(name)
 results.append(result)

In [448]:
#what is going on here
for i in range(len(names)):
    print(names[i],results[i].mean())

Random Forest Regressor 0.1582693447133351
Linear Regression 0.10549757614632911
Support Vector Regression -0.0397744913841358


In [449]:
algo = RandomForestRegressor()
model = algo.fit(X_train,y_train)
print(cross_val_score(model, X_train, y_train)) 
model.score(X_train,y_train)

[0.15191057 0.15750137 0.16250847 0.16190149 0.13910014]


0.2293345033287022

In [450]:
algo = LinearRegression()
model = algo.fit(X_train,y_train)
print(cross_val_score(model, X_train, y_train)) 
model.score(X_train, y_train)

[0.10943976 0.10916768 0.12191145 0.10956625 0.09992774]


0.11103196052744169

In [451]:
algo = SVR()
model = algo.fit(X_train,y_train)
print(cross_val_score(model, X_train, y_train)) 
model.score(X_train,y_train)

[-0.05197082 -0.03900908 -0.04336682 -0.04004579 -0.0479202 ]


-0.04415655004371288